# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686587


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:07<01:09,  2.57s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:50,  1.95s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:36,  1.46s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:26,  1.09s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.36it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  2.83it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  3.12it/s]

Rendering models: 100%|██████████| 30/30 [00:16<00:00,  2.54it/s]

Anfrily                                0.000205
smithkurtis                            0.000324
not-logged-in-975ec4b89e5ce0be836b     0.000574
Camelron                               0.000129
micaroe                                0.000160
arjhun                                 0.075214
Dynamatt                               0.000143
CScampstudent                          0.043677
Enfysh                                 0.019263
Petrusperes                            0.000612
not-logged-in-f109601c42830b27689b     0.370611
sn346808                               0.022522
JaceSu                                 0.000107
not-logged-in-14d09f70263c1fe15cc3     0.000384
caseyroberto                           0.000123
Tenorclef                              0.001803
Linda_J._Berkel                        0.000129
1wsm1                                  0.001126
not-logged-in-2eb38841187e4efc56ae    57.983094
Mongin                                 0.000392
not-logged-in-53d8da56a55dac1b32ed     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())